![JohnSnowLabs](https://sparknlp.org/assets/images/logo.png)

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/JohnSnowLabs/spark-nlp/blob/master/examples/python/transformers/onnx/HuggingFace_ONNX_in_Spark_NLP_MPNetForTokenClassification.ipynb)

## Import ONNX MPNetForTokenClassification models from HuggingFace 🤗  into Spark NLP 🚀

Let's keep in mind a few things before we start 😊

- ONNX support was introduced in  `Spark NLP 5.0.0`, enabling high performance inference for models.
- `MPNetForTokenClassification` is only available since in `Spark NLP 5.3.1` and after. So please make sure you have upgraded to the latest Spark NLP release
- You can import MPNet models trained/fine-tuned for token classification via `MPNetForTokenClassification` or `TFMPNetForTokenClassification`. These models are usually under `Token Classification` category and have `mpnet` in their labels
- Reference: [TFMPNetForTokenClassification](https://huggingface.co/docs/transformers/en/model_doc/mpnet#transformers.TFMPNetForTokenClassification)
- Some [example models](https://huggingface.co/models?filter=mpnet&pipeline_tag=token-classification)

## Export and Save HuggingFace model

- Let's install `transformers` package with the `onnx` extension and it's dependencies. You don't need `onnx` to be installed for Spark NLP, however, we need it to load and save models from HuggingFace.
- We lock `transformers` on version `4.52.4`. This doesn't mean it won't work with the future releases
- Albert uses SentencePiece, so we will have to install that as well

In [1]:
!pip install -q --upgrade transformers[onnx]==4.52.4 optimum onnx

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 424.6/424.6 kB 18.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.6/17.6 MB 59.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.4/16.4 MB 75.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.7/212.7 kB 17.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 2.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 73.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 66.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 45.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 13.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

- HuggingFace has an extension called Optimum which offers specialized model inference, including ONNX. We can use this to import and export ONNX models with `from_pretrained` and `save_pretrained`.
- We'll use [hf-tiny-model-private/tiny-random-MPNetForTokenClassification](https://huggingface.co/hf-tiny-model-private/tiny-random-MPNetForTokenClassification) model from HuggingFace as an example
- In addition to `TFMPNetForTokenClassification` we also need to save the `MPNetTokenizer`. This is the same for every model, these are assets needed for tokenization inside Spark NLP.

In [2]:
from transformers import MPNetTokenizer
from optimum.onnxruntime import ORTModelForTokenClassification

MODEL_NAME = 'hf-tiny-model-private/tiny-random-MPNetForTokenClassification'
EXPORT_PATH = f"onnx_models/{MODEL_NAME}"

ort_model = ORTModelForTokenClassification.from_pretrained(MODEL_NAME, export=True)
ort_model.save_pretrained(EXPORT_PATH)

tokenizer = MPNetTokenizer.from_pretrained(MODEL_NAME)
tokenizer.save_pretrained(EXPORT_PATH)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/635 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/957k [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/1.43k [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/4.68k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/22.3k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/938k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/280 [00:00<?, ?B/s]

('onnx_models/hf-tiny-model-private/tiny-random-MPNetForTokenClassification/tokenizer_config.json',
 'onnx_models/hf-tiny-model-private/tiny-random-MPNetForTokenClassification/special_tokens_map.json',
 'onnx_models/hf-tiny-model-private/tiny-random-MPNetForTokenClassification/vocab.txt',
 'onnx_models/hf-tiny-model-private/tiny-random-MPNetForTokenClassification/added_tokens.json')

Let's have a look inside these two directories and see what we are dealing with:

In [4]:
!ls -l {EXPORT_PATH}

total 1032
-rw-r--r-- 1 root root     562 Jun 25 11:09 config.json
-rw-r--r-- 1 root root 1033389 Jun 25 11:09 model.onnx
-rw-r--r-- 1 root root     964 Jun 25 11:09 special_tokens_map.json
-rw-r--r-- 1 root root    1469 Jun 25 11:09 tokenizer_config.json
-rw-r--r-- 1 root root    4682 Jun 25 11:09 vocab.txt


- We need to convert `vocab.json` to a plain `vocab.txt` format, as required by Spark NLP, then move it to an `assets` folder.
- We also need `labels` and their `ids` which is saved inside the model's config. We will save this inside `labels.txt`

In [10]:
import json

!mkdir -p {EXPORT_PATH}/assets && mv {EXPORT_PATH}/vocab.txt {EXPORT_PATH}/assets

with open(f"{EXPORT_PATH}/assets/labels.txt", "w") as f:
    f.write("\n".join(ort_model.config.id2label[k] for k in sorted(ort_model.config.id2label)))

In [13]:
!ls -lR {EXPORT_PATH}

onnx_models/hf-tiny-model-private/tiny-random-MPNetForTokenClassification:
total 1028
drwxr-xr-x 2 root root    4096 Jun 25 11:11 assets
-rw-r--r-- 1 root root     562 Jun 25 11:09 config.json
-rw-r--r-- 1 root root 1033389 Jun 25 11:09 model.onnx
-rw-r--r-- 1 root root     964 Jun 25 11:09 special_tokens_map.json
-rw-r--r-- 1 root root    1469 Jun 25 11:09 tokenizer_config.json

onnx_models/hf-tiny-model-private/tiny-random-MPNetForTokenClassification/assets:
total 12
-rw-r--r-- 1 root root   15 Jun 25 11:11 labels.txt
-rw-r--r-- 1 root root 4682 Jun 25 11:09 vocab.txt


Voila! We have our `vocab.txt` and `labels.txt` inside assets directory

## Import and Save DeBertaForTokenClassification in Spark NLP


- Install and set up Spark NLP in Google Colab
- This example uses specific versions of `pyspark` and `spark-nlp` that have been tested with the transformer model to ensure everything runs smoothly.

In [14]:
!pip install -q pyspark==3.5.4 spark-nlp==5.5.3

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 317.3/317.3 MB 4.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 635.7/635.7 kB 46.1 MB/s eta 0:00:00


Let's start Spark with Spark NLP included via our simple `start()` function

In [15]:
import sparknlp

spark = sparknlp.start()

print("Spark NLP version: ", sparknlp.version())
print("Apache Spark version: ", spark.version)

Spark NLP version:  5.5.3
Apache Spark version:  3.5.4


- Let's use `loadSavedModel` functon in `MPNetForTokenClassification` which allows us to load TensorFlow model in SavedModel format
- Most params can be set later when you are loading this model in `MPNetForTokenClassification` in runtime like `setMaxSentenceLength`, so don't worry what you are setting them now
- `loadSavedModel` accepts two params, first is the path to the TF SavedModel. The second is the SparkSession that is `spark` variable we previously started via `sparknlp.start()`
- NOTE: `loadSavedModel` accepts local paths in addition to distributed file systems such as `HDFS`, `S3`, `DBFS`, etc. This feature was introduced in Spark NLP 4.2.2 release. Keep in mind the best and recommended way to move/share/reuse Spark NLP models is to use `write.save` so you can use `.load()` from any file systems natively.st and recommended way to move/share/reuse Spark NLP models is to use `write.save` so you can use `.load()` from any file systems natively.



In [17]:
from sparknlp.annotator import MPNetForTokenClassification

tokenClassifier = MPNetForTokenClassification.loadSavedModel(
     EXPORT_PATH,
     spark
 )\
 .setInputCols(["document",'token'])\
 .setOutputCol("ner")\
 .setCaseSensitive(True)\
 .setMaxSentenceLength(128)

- Let's save it on disk so it is easier to be moved around and also be used later via `.load` function

In [18]:
tokenClassifier.write().overwrite().save("./{}_spark_nlp_onnx".format(EXPORT_PATH))

Let's clean up stuff we don't need anymore

In [19]:
!rm -rf {EXPORT_PATH}

Awesome 😎  !

This is your MPNetForTokenClassification model from HuggingFace 🤗  loaded and saved by Spark NLP 🚀

In [20]:
! ls -l {EXPORT_PATH}_spark_nlp_onnx

total 1020
drwxr-xr-x 4 root root    4096 Jun 25 11:14 fields
drwxr-xr-x 2 root root    4096 Jun 25 11:14 metadata
-rw-r--r-- 1 root root 1033683 Jun 25 11:14 mpnet_classification_onnx


Now let's see how we can use it on other machines, clusters, or any place you wish to use your new and shiny DeBertaForTokenClassification model 😊

In [21]:
tokenClassifier_loaded = MPNetForTokenClassification.load("./{}_spark_nlp_onnx".format(EXPORT_PATH))\
  .setInputCols(["document",'token'])\
  .setOutputCol("ner")

You can see what labels were used to train this model via `getClasses` function:

In [22]:
tokenClassifier_loaded.getClasses()

['LABEL_1', 'LABEL_0']

This is how you can use your loaded classifier model in Spark NLP 🚀 pipeline:

In [23]:
from sparknlp.base import DocumentAssembler
from sparknlp.annotator import Tokenizer
from pyspark.ml import Pipeline

document_assembler = DocumentAssembler() \
    .setInputCol('text') \
    .setOutputCol('document')

tokenizer = Tokenizer() \
    .setInputCols(['document']) \
    .setOutputCol('token')

pipeline = Pipeline(stages=[
    document_assembler,
    tokenizer,
    tokenClassifier_loaded
])

example = spark.createDataFrame([["I love spicy food"]]).toDF("text")

model = pipeline.fit(example)
result = model.transform(example)

result.select("text", "ner.result").show(truncate=False)

+-----------------+------------------------------------+
|text             |result                              |
+-----------------+------------------------------------+
|I love spicy food|[LABEL_0, LABEL_0, LABEL_0, LABEL_1]|
+-----------------+------------------------------------+



That's it! You can now go wild and use hundreds of `MPNetForTokenClassification` models from HuggingFace 🤗 in Spark NLP 🚀
